In [2]:
import os
from PIL import Image
import pandas as pd
import numpy as np
import random

## 生成模版

In [3]:
# 设置素材参数
W = 400
H = 400
EXTENSION = 'png'
PARTS = "./parts"
IMAGES = './images'

In [4]:
# 遍历文件
files_path = []
for root, dirs, _ in os.walk(PARTS):
    if root != PARTS:
        for _, _, files in os.walk(root):
            for file in files:
                if file != ".DS_Store":
                    files_path.append(os.path.join(root,file))
files_path

['./parts/0_Background/blue.png',
 './parts/0_Background/green.png',
 './parts/0_Background/white.png',
 './parts/0_Background/red.png',
 './parts/0_Background/purple.png',
 './parts/2_First Latter/K.png',
 './parts/2_First Latter/J.png',
 './parts/2_First Latter/H.png',
 './parts/2_First Latter/I.png',
 './parts/2_First Latter/B.png',
 './parts/2_First Latter/C.png',
 './parts/2_First Latter/D.png',
 './parts/2_First Latter/E.png',
 './parts/2_First Latter/G.png',
 './parts/2_First Latter/F.png',
 './parts/1_Second Letter/Z.png',
 './parts/1_Second Letter/Y.png',
 './parts/1_Second Letter/X.png',
 './parts/1_Second Letter/T.png',
 './parts/1_Second Letter/V.png',
 './parts/1_Second Letter/A.png',
 './parts/1_Second Letter/W.png',
 './parts/1_Second Letter/R.png',
 './parts/1_Second Letter/p.png',
 './parts/1_Second Letter/Q.png']

In [5]:
# 验证图片格式和尺寸
for path in files_path:
    assert path.split('.')[-1] == EXTENSION, f"{path} 的扩展名不是 {EXTENSION} " 
    im = Image.open(path)
    w, h = im.size
    assert w == W, f'{path} width not equal {W}'
    assert h == H, f'{path} height not equal {H}'

In [6]:
# 导出表格
attrs = [path.split('/')[2:] for path in files_path]
d = {'prop':[a[0].split('_')[1] for a in attrs], 'value': [a[1][:-4] for a in attrs],'ratio': 1}
df = pd.DataFrame(data=d)
df.to_csv('ratio.csv', index=False)

## 生成图片

In [7]:
df_csv = pd.read_csv('./ratio.csv')
df_group = df_csv.groupby(['prop','value']).agg({'ratio': 'sum'})
df_pac = df_group.groupby(level=0).apply(lambda x: x / float(x.sum()))
df_pac

ratio
prop          value        
Background    blue      0.2
              green     0.2
              purple    0.2
              red       0.2
              white     0.2
First Latter  B         0.1
              C         0.1
              D         0.1
              E         0.1
              F         0.1
              G         0.1
              H         0.1
              I         0.1
              J         0.1
              K         0.1
Second Letter A         0.1
              Q         0.1
              R         0.1
              T         0.1
              V         0.1
              W         0.1
              X         0.1
              Y         0.1
              Z         0.1
              p         0.1

In [218]:
# 随机选择生成属性
props = df_csv['prop'].unique()

def random_attr():
    attributes = []
    for prop in props:
        k = random.random()
        ratio_arr = df_pac.query(f"prop == '{prop}'").ratio.values
        cum_arr = np.cumsum(ratio_arr) - k
        first_index = next(x[0] for x in enumerate(cum_arr) if x[1] > 0)
        value = df_pac.loc[(prop), :].index[first_index]
        attributes.append({"value" :value, "trait_type": prop})
    return attributes

In [212]:
import json

AMOUNT = 11
used_attributes = []

df_attr = pd.DataFrame(columns = ['ipfshash', 'path'] + [i['trait_type'] for i in attributes])

# 检验AMOUNT是否合理
assert np.min(df_pac['ratio']) * AMOUNT > 1, '总数不合理，应该增加属性素材或者减少总量'

## 如果文件夹有png报错
assert len(list(filter(lambda f:f.split('.')[1]=="png",os.listdir(IMAGES)))) == 0,\
    f'{IMAGES} 文件夹不为空，先备份原数据和表格'
    
for i in range(AMOUNT):
    # 避免重复
    attributes = random_attr()
    while attributes in used_attributes:
        attributes = random_attr()
    used_attributes.append(attributes)
    # 按叠加顺序获取要读取的图片
    sorted_paths = sorted([next(path for path in files_path if attr['trait_type']+'/'+attr['value'] in path) 
                            for attr in attributes])
    base_img = Image.new('RGBA', (W,H), (0, 0, 0, 0))
    for path in sorted_paths:
        img = Image.open(path, 'r')
        base_img.paste(img, (0,0), mask=img) 
    # 保存图片
    base_img.save(os.path.join(IMAGES,str(i)+'.png')) 
    # 添加属性
    df_attr = df_attr.append(
        { 'ipfshash' : None, 'path' : os.path.join(IMAGES,str(i)+'.png') }
        | {i['trait_type']:i['value'] for i in attributes}, 
        ignore_index = True
    )
    
df_attr.to_csv(os.path.join(IMAGES,'attr.csv'), index_label='id')

## 重命名图片

In [213]:
# 通过计算hash去除画面一样的图
import hashlib, os
file_list = list(filter(lambda f:f.split('.')[1]=="png",os.listdir(IMAGES)))
duplicates = []
hash_keys = dict()
for index, filename in enumerate(file_list): 
    with open(os.path.join(IMAGES,filename), 'rb') as f:
        filehash = hashlib.md5(f.read()).hexdigest()
    if filehash not in hash_keys: 
        hash_keys[filehash] = index
    else:
        duplicates.append((index,hash_keys[filehash]))

for index in duplicates:
    os.remove(os.path.join(IMAGES,file_list[index[0]]))

In [219]:
# 支持手动删除部分图片

In [220]:
# 重命名文件和csv
def rename_files(root, start_index=0):
    df = pd.read_csv(os.path.join(root, 'attr.csv'))
    exist_paths = [os.path.join(root,r) for r in list(filter(lambda f:f.split('.')[1]=="png",os.listdir(root)))]
    new_df = df[df['path'].isin(exist_paths)]
    
    for index,(idx,row) in enumerate(new_df.iterrows()):
        old_name = row.values[2]
        new_name = f'./images/{index}.png'
        new_df.loc[idx,'id'] = index
        new_df.loc[idx,'path'] = new_name
        os.rename(old_name, new_name)
    
    new_df = new_df.set_index('id')
    new_df.to_csv(os.path.join(root,'attr.csv'))
       
rename_files(IMAGES)